# GPT-4 API: Zero-Shot Learning with Prompt Engineering
## ACL 2023 Conference
## WASSA 2023 Shared Task on Empathy, Emotion, and Personality Detection in Interactions
More details [here](https://codalab.lisn.upsaclay.fr/competitions/11167#learn_the_details)

In [1]:
import openai
import os
import re
import numpy as np
import pandas as pd
import time
import tiktoken
import backoff
from typing import List
from sklearn.metrics import classification_report, multilabel_confusion_matrix

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 400)

import warnings
warnings.filterwarnings("ignore")

# to see all env variables:
#for name, value in os.environ.items():
#    print("{0}: {1}".format(name, value))

In [2]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    '''Return number of tokens used in a list of messages for ChatGPT'''
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        #print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo":
        #print("Warning: gpt-3.5-turbo may change over time. Returning num tokens assuming gpt-3.5-turbo-0301.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301")
    elif model == "gpt-4":
        #print("Warning: gpt-4 may change over time. Returning num tokens assuming gpt-4-0314.")
        return num_tokens_from_messages(messages, model="gpt-4-0314")
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif model == "gpt-4-0314":
        tokens_per_message = 3
        tokens_per_name = 1
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [3]:
random_state = 47

# Load data

In [4]:
file1    = 'data/df_train.pkl'
df_train = pd.read_pickle(file1)

file2    = 'data/df_dev.pkl'
df_dev   = pd.read_pickle(file2)

print(df_train.shape, df_dev.shape)

2023-05-09 17:31:09.925130: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(792, 36) (208, 38)


In [5]:
cols = [ 'essay_clean_spellchecked', 'emotion', ]
with pd.option_context('display.max_colwidth', None):
    display(df_dev[cols].head(25))

,essay_clean_spellchecked,emotion
0,"How sad is it that this kind of pain and suffering, and those kind of living conditions still exist today? what a gap we have in society between developed countries and those that aren't. It's crazy to drive around the US and see all the money people spend on pointless things, and then to think about how the people in Haiti are living.",[Sadness]
1,The article is kind of tragic and hits close to home as I am the son of Haitian immigrants. Haiti has a lot of problems that only become exaggerated during natural disasters. I think what the Haitian people really need from the international community is help developing infrastructure so they can address these issues themselves. Foreign aid only acts as a band aid.,[Sadness]
2,"I think that these kinds of stories, are sad, yet inspirational and leave you with kind of a good feeling. Even though his story is sad, it's cool and inspiring/motivational to see that he rose up against his circumstances. That he worked hard to make something of himself and he succeeded in what he wanted to do.",[Sadness]
3,It's crazy that random accidents like this happen everyday. I am not a baseball fan but of course enjoy a baseball game every now and again. I lived and worked in Miami too so I am vaguely familiar with that baseball player who unfortunately passed away. The effort to save him was great but unfortunately bad things seem to happen every day. He was so young too so it makes it worse.,[Neutral]
4,"This story makes me so so sad.... As someone who also grew up in the system, I can strongly relate. It's sad that America has not figured out a better and more safe system to handle kid's without parents or with parents who are unfit. A lot of the times, the system is no better, or even worse than the situation kids were in before, and I think this story is a good example of that.",[Sadness]
5,"After reading the article, my first reaction and feeling is that i feel really bad for the brothers. I feel like people their age should not have to be locked inside a jail cell. They should be out in the world improving themselves and being normal people. It's also really sad for the family members of these brothers as well because they are probably all suffering and worrying.",[Sadness]
6,"I didn't know coal mining had such adverse effects on the surrounding environment. It has basically ruined the lives of the people who live nearby these mines. And the animal populations too, imagine a heard of elephants not able to sustain themselves with the food available and needing to invade human territory...They must really be in a desperate situation.",[Neutral]
7,"This is very sad. I can't imagine having elephants come stampede my house in the middle of the night. What a terrible and sad situation, and these poor people can't even do anything about it. Someone needs to stop the deforestation and stop polluting the air these people breathe, it is not right that they are doing and all for the sake of turning a profit.",[Sadness]
8,"Guys, reading this article really hits home for me. If you or someone you know is having suicidal thoughts, please get help from the available sources. Suicide is no joke and it is a shame when someone does not get the help they need. I've struggled with this for a few years now but I got the help I needed. This woman was not as fortunate.",[Sadness]
9,Hey guys. So I just read this article about Iraqi Christians being persecuted by Muslims in Iraq. I don't understand why people of different religious backgrounds can't get along there. I'm sure it is a cultural thing but it is such unnecessary violence and conflict. It hurts both sides and I wish there was a way we could get them to set aside their differences. But not military action. We don't need another war.,[Neutral]


# Prompts

In [7]:
# categories are listed in the decreasing frequency order
prompt_one = """1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with \
tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this \
emotion and nothing elser
2 - You can add another emotion from the above list, but only if it is also relevant. If there are two relevant \
emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.
3 - Do not add the second emotion if it doesn't have a significant semantic weight in the text below.

Text: ```{}```
"""
s = 'This is a text sample'
print(prompt_one.format(s), '\n')

1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but only if it is also relevant. If there are two relevant emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.
3 - Do not add the second emotion if it doesn't have a significant semantic weight in the text below.

Text: ```This is a text sample```
 



In [8]:
# Using followup Q1 can improve the reponse. If the reponse has multiple words, first parse it and try to find
# the category in it. Only if this doesn't work, send followup Q2. ChatGPT can offer the second category in reponse
# to Q1, but can change its mind again and offer a third category if asked Q2
followup = """Are you sure about that? If yes, output the same emotion; if no, change the emotion"""
followup

'Are you sure about that? If yes, output the same emotion; if no, change the emotion'

# Sample request

In [9]:
#openai.api_key = "sk-..."
openai.api_key = os.getenv("OPENAI_API_KEY")
model          = 'gpt-4'
labels_set     = {'sadness', 'neutral', 'anger', 'disgust', 'hope', 'fear', 'surprise', 'joy'} 
clean          = re.compile(r'[^a-zA-Z ]+')
multi_spaces   = re.compile('\s{2,}')

In [10]:
def verify_label(label_):
    '''
       Verify if label_ contains any of the categories
       from the predefined set of labels
    '''
    label_ = clean.sub(' ', label_)
    label_ = multi_spaces.sub(' ', label_).lower().split()
    res    = [i.capitalize() for i in label_ if i in labels_set]
    res    = list(set(res))
    return '/'.join(sorted(res)) if res else None

In [11]:
def verify_num_tokens(model, messages):
    '''Check that there is enough tokens available for a ChatGPT repsonse'''
    num_tokens_tiktoken = num_tokens_from_messages(messages, model)
    if num_tokens_tiktoken > 7500:
        print(f'Number of tokens is {num_tokens_tiktoken} which exceeds 3950')
        print(f'TEXT: {text_}\n')
        return False
    else:
        return True


@backoff.on_exception(backoff.expo, openai.error.RateLimitError, max_time=10)
def get_response(model, messages, temperature=0, max_tokens=None):
    '''Send request, return reponse'''
    print('\nTempreature:', temperature, '\n')
    response  = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature,        # range(0,2), the more the less deterministic / focused
        #top_p = 1,                        # top probability mass, e.g. 0.1 = only tokens from top 10% proba mass
        #n = 1,                            # number of chat completions
        #max_tokens = max_tokens,          # tokens to return
        #stream = False,        
        #stop=None,                        # sequence to stop generation (new line, end of text, etc.)
        )
    content = response['choices'][0]['message']['content'].strip()
    #num_tokens_api = response['usage']['prompt_tokens']
    return content

In [12]:
def classify_text(text_, prompt_):
    '''Classify text_ using prompt_ and GPT-4 API'''
        
    print('Prompt:\n', prompt_)
    # compose messages and check num_tokens
    messages = [
            #{ "role": "system", "content": "You are a helpful emotion classifier.", },
            { "role": "user", "content": prompt_.format(text_), },
            ]
    #if not verify_num_tokens(model, messages): return None
    label_    = get_response(model, messages, )
    old_label = label_
    label_    = verify_label(label_)        # get just the category if response is too long
    print(f"First iteration: '{old_label}' '{label_}'.")
        
    # if label not found in response text - second, extended chat
    if label_ is None:
        print(f'\tAsking the clarifying question')
        messages += [
            { "role": "assistant", "content": old_label, },
            { "role": "user", "content": followup, }
            ]        
        label_    = get_response(model, messages, )
        old_label = label_
        label_    = verify_label(label_)        # get just the category if response is too long
        print(f"\tSecond iteration: '{old_label}' '{label_}' ")
            
    return label_ if label_ is not None else old_label

In [14]:
def classify_text_with_clarifying(text_, prompt_):
    '''
       Classify text_ using prompt_ and GPT-4 API,
       then clarify response with followup question -
       this can help make the response more precise
    '''
        
    # compose messages and check num_tokens
    messages = [
            { "role": "system", "content": "You are a helpful text classifier.", },
            { "role": "user", "content": prompt_.format(text_), },
            ]
    #if not verify_num_tokens(model, messages): return None
    label_    = get_response(model, messages)
    old_label = label_
    label_    = verify_label(label_)                      # get just the category if response is too long
        
    # ask additional clarifying question - sometimes it helps
    messages += [
        { "role": "assistant", "content": old_label, },
        { "role": "user", "content": followup, }
        ]
    #time.sleep( random.uniform(1.1, 1.8) )                # wait not to overload ChatGPT
    label2_    = get_response(model, messages)
    old_label2 = label2_
    label2_    = verify_label(label2_)                    # get just the category if response is too long

    return old_label, label_, old_label2, label2_

In [15]:
idx = 2
text, groundtruth_labels = df_dev[['essay_clean_spellchecked', 'emotion']].values[idx]
label = classify_text(text, prompt_one)
messages = [
    {'role': 'user', 'content': prompt_one.format(text)},
]

print(prompt_one.format( text ))
print(f"\nGROUNDTRUTH LABEL:\n{'/'.join( groundtruth_labels )}")
print(f"\nPREDICTED LABEL:\n{label}")
print(f'\nTOTAL TOKENS: { num_tokens_from_messages(messages, model) }')

Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but only if it is also relevant. If there are two relevant emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.
3 - Do not add the second emotion if it doesn't have a significant semantic weight in the text below.

Text: ```{}```


Tempreature: 0 

First iteration: 'Hope/Sadness' 'Hope/Sadness'.
1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but only if it is also relevant. If there are two relevant em

# Batch Request

In [16]:
# target variables
label2key = {   
    'Anger':    0,
    'Disgust':  1,
    'Fear':     2,
    'Hope':     3,    
    'Joy':      4,
    'Neutral':  5,
    'Sadness':  6,
    'Surprise': 7,
}
key2label = {v: k for k,v in label2key.items()}
print(key2label)

{0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Hope', 4: 'Joy', 5: 'Neutral', 6: 'Sadness', 7: 'Surprise'}


In [18]:
def get_target(emotions: List[str])->List[int]:
    '''
        Convert list of strings with categories into list of 0s and 1s with length 8 because there are 8 categories;
        1 in the i-th position means that this essay belongs to the i-th category as in key2label[i]
    '''
    res  = [0]*8
    idxs = [label2key[e] for e in emotions]    
    for idx in idxs:
        res[idx] = 1
    return res

In [19]:
df_dev.head()

,article_id,conversation_id,speaker_number,essay_id,speaker_id,essay,essay_clean,split,gender,education,race,age,income,emotion,emotion_count,char_length,word_length,target_encoded,article,article_clean,essay_clean_docs,essay_clean_spellchecked,article_clean_docs,article_clean_spellchecked,compare1,compare2,gpt_embedding,closest_texts,emotion_no_2nd_neut,gpt35_keywords,gpt35_title,gpt35_summary,gpt4_title,gpt4_summary,gpt4_keywords,davinci_preds2,davinci_preds_converted2,davinci_preds_encoded2
0,35,1,1,0,68,How sad is it that this kind of pain and suffe...,How sad is it that this kind of pain and suffe...,dev,2,2,1,21,20000,[Sadness],1,339,63,"[0, 0, 0, 0, 0, 0, 1, 0]","A month after Hurricane Matthew, 800,000 Haiti...","A month after Hurricane Matthew, 800,000 Haiti...","(How, sad, is, it, that, this, kind, of, pain,...",How sad is it that this kind of pain and suffe...,"(A, month, after, Hurricane, Matthew, ,, 800,0...","A month after Hurricane Matthew, 800,000 Haiti...",False,False,"[0.01686943881213665, -0.00237144879065454, 0....",[This situation in Haiti is terrible and sadde...,[Sadness],"pain, suffering, living conditions, developed ...",The Disparity Between Developed Countries and ...,The author expresses sadness over the existenc...,The Stark Contrast Between Wealth and Poverty ...,The text highlights the disparity between deve...,"sad, pain, suffering, living conditions, exist...","[[0.8797003859451549, Sadness], [0.07585283480...","[Sadness, Disgust]","[0, 1, 0, 0, 0, 0, 1, 0]"
1,35,4,1,3,79,The article is kind of tragic and hits close t...,The article is kind of tragic and hits close t...,dev,1,6,3,33,64000,[Sadness],1,367,63,"[0, 0, 0, 0, 0, 0, 1, 0]","A month after Hurricane Matthew, 800,000 Haiti...","A month after Hurricane Matthew, 800,000 Haiti...","(The, article, is, kind, of, tragic, and, hits...",The article is kind of tragic and hits close t...,"(A, month, after, Hurricane, Matthew, ,, 800,0...","A month after Hurricane Matthew, 800,000 Haiti...",True,False,"[-0.007789629977196455, 0.011638623662292957, ...","[In the 21st century, people are still starvin...",[Sadness],"Haitian immigrants, Haiti, problems, natural d...","""Developing Infrastructure: The Key to Helping...","The author, who is the son of Haitian immigran...",Haitian Infrastructure Development: A Long-Ter...,The text discusses the tragic situation in Hai...,"article, tragic, close to home, son, Haitian i...","[[0.8202061810703213, Neutral], [0.16278792482...","[Neutral, Sadness]","[0, 0, 0, 0, 0, 1, 1, 0]"
2,213,7,1,6,68,"I think that these kinds of stories, are sad, ...","I think that these kinds of stories, are sad, ...",dev,2,2,1,21,20000,[Sadness],1,315,57,"[0, 0, 0, 0, 0, 0, 1, 0]",Miami Marlins star pitcher Jose Fernandez kill...,Miami Marlins star pitcher Jose Fernandez kill...,"(I, think, that, these, kinds, of, stories, ,,...","I think that these kinds of stories, are sad, ...","(Miami, Marlins, star, pitcher, Jose, Fernande...",Miami Marlins star pitcher Jose Fernandez kill...,True,False,"[-0.004882862325757742, -0.016483262181282043,...",[It is so sad that someone who had such an ama...,[Sadness],"stories, sad, inspirational, good feeling, coo...",The Inspiring Story of Overcoming Circumstances,The text discusses how stories that are sad bu...,Overcoming Adversity: The Power of Inspiration...,The text describes a story as sad but also ins...,"sad, inspirational, good feeling, story, cool,...","[[0.6857936968091514, Sadness], [0.10173794890...","[Sadness, Joy]","[0, 0, 0, 0, 1, 0, 1, 0]"
3,213,9,1,8,84,It's crazy that random accidents like this hap...,It's crazy that random accidents like this hap...,dev,2,4,1,25,55000,[Neutral],1,385,72,"[0, 0, 0, 0, 0, 1, 0, 0]",Miami Marlins star pitcher Jose Fernandez kill...,Miami Marlins star pitcher Jose Fernandez kill...,"(It, 's, crazy, that, random, accidents, like,...",It's crazy that random accidents like this hap...,"(Miami, Marlins, star, pitcher, Jose, Fernande...",Miami Marlins st

In [20]:
start = time.time()
res   = dict()
count = 0
for text in df_dev['essay_clean_spellchecked'].values:
    if text in res:
        continue
    try:
        res[ text ] = classify_text(text, prompt_one)
    except Exception as e:
        print(f'\nText: {text}\nError: {e}\n')
                
    count += 1    
    if count % 10 == 0:
        print(f'Processing text {count}')                        
        
elapsed = (time.time() - start)/60
print(f'\nTime elapsed {round(elapsed, 4)} min')

Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but only if it is also relevant. If there are two relevant emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.
3 - Do not add the second emotion if it doesn't have a significant semantic weight in the text below.

Text: ```{}```


Tempreature: 0 

First iteration: 'Sadness/Hope' 'Hope/Sadness'.
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but only if it is also relevant. If there are two re

First iteration: 'Anger/Sadness' 'Anger/Sadness'.
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but only if it is also relevant. If there are two relevant emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.
3 - Do not add the second emotion if it doesn't have a significant semantic weight in the text below.

Text: ```{}```


Tempreature: 0 

First iteration: 'Hope/Sadness' 'Hope/Sadness'.
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, bu

First iteration: 'Sadness/Disgust' 'Disgust/Sadness'.
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but only if it is also relevant. If there are two relevant emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.
3 - Do not add the second emotion if it doesn't have a significant semantic weight in the text below.

Text: ```{}```


Tempreature: 0 

First iteration: 'Disgust/Sadness' 'Disgust/Sadness'.
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the abov

First iteration: 'Disgust/Joy' 'Disgust/Joy'.
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but only if it is also relevant. If there are two relevant emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.
3 - Do not add the second emotion if it doesn't have a significant semantic weight in the text below.

Text: ```{}```


Tempreature: 0 

First iteration: 'Sadness/Neutral' 'Neutral/Sadness'.
Processing text 40
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion fr

First iteration: 'Sadness' 'Sadness'.
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but only if it is also relevant. If there are two relevant emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.
3 - Do not add the second emotion if it doesn't have a significant semantic weight in the text below.

Text: ```{}```


Tempreature: 0 

First iteration: 'Sadness/Hope' 'Hope/Sadness'.
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but only if it

...

First iteration: 'Hope/Joy' 'Hope/Joy'.
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but only if it is also relevant. If there are two relevant emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.
3 - Do not add the second emotion if it doesn't have a significant semantic weight in the text below.

Text: ```{}```


Tempreature: 0 

First iteration: 'Neutral/Sadness' 'Neutral/Sadness'.
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but on

First iteration: 'Sadness/Hope' 'Hope/Sadness'.
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but only if it is also relevant. If there are two relevant emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.
3 - Do not add the second emotion if it doesn't have a significant semantic weight in the text below.

Text: ```{}```


Tempreature: 0 

First iteration: 'Sadness/Neutral' 'Neutral/Sadness'.
Processing text 170
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion

First iteration: 'Fear/Hope' 'Fear/Hope'.
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but only if it is also relevant. If there are two relevant emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.
3 - Do not add the second emotion if it doesn't have a significant semantic weight in the text below.

Text: ```{}```


Tempreature: 0 

First iteration: 'Disgust/Sadness' 'Disgust/Sadness'.
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but 

First iteration: 'Anger/Disgust' 'Anger/Disgust'.
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but only if it is also relevant. If there are two relevant emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.
3 - Do not add the second emotion if it doesn't have a significant semantic weight in the text below.

Text: ```{}```


Tempreature: 0 

First iteration: 'Disgust/Sadness' 'Disgust/Sadness'.
Prompt:
 1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above li

First iteration: 'Sadness/Fear' 'Fear/Sadness'.

Time elapsed 7.2288 min


In [ ]:
len(res)

### If a followup question was not used

In [21]:
# if one label is the output
df_dev['pred_all'] = df_dev['essay_clean_spellchecked'].map( res )
print(df_dev.isna().sum())
print(df_dev['pred_all'].value_counts())

article_id                    0
conversation_id               0
speaker_number                0
essay_id                      0
speaker_id                    0
essay                         0
essay_clean                   0
split                         0
gender                        0
education                     0
race                          0
age                           0
income                        0
emotion                       0
emotion_count                 0
char_length                   0
word_length                   0
target_encoded                0
article                       0
article_clean                 0
essay_clean_docs              0
essay_clean_spellchecked      0
article_clean_docs            0
article_clean_spellchecked    0
compare1                      0
compare2                      0
gpt_embedding                 0
closest_texts                 0
emotion_no_2nd_neut           0
gpt35_keywords                0
gpt35_title                   0
gpt35_su

In [45]:
print(df_dev[ df_dev['pred_all']=="Yes, I'm sure. The category is Disappointment."])
df_dev.at[ 54, 'pred_all' ] = 'Fear'
df_dev.at[ 74, 'pred_all' ] = 'Neutral'
df_dev.at[ 26, 'pred_all' ] = 'Sadness'

    article_id  conversation_id  speaker_number  essay_id  speaker_id  \
26         233               93               1        92          71   

                                                essay  \
26  Unfortunately in countries like these the offi...   

                                          essay_clean split  gender  \
26  Unfortunately in countries like these the offi...   dev       1   

    education  race  age  income    emotion  emotion_count  char_length  \
26          6     2   32   35000  [Disgust]              1          369   

    word_length            target_encoded  \
26           72  [0, 1, 0, 0, 0, 0, 0, 0]   

                                              article  \
26  Nigeria investigates reports that officials ra...   

                                        article_clean  \
26  Nigeria investigates reports that officials ra...   

                                     essay_clean_docs  \
26  (Unfortunately, in, countries, like, these, th...   

        

In [22]:
df_dev['pred_all'].value_counts()

Hope/Sadness        54
Disgust/Sadness     28
Fear/Sadness        24
Anger/Sadness       17
Anger/Disgust       14
Neutral/Sadness     11
Fear/Hope           10
Sadness              9
Disgust/Fear         9
Disgust/Hope         4
Hope/Neutral         4
Anger/Hope           3
Fear/Surprise        2
Disgust/Neutral      2
Hope/Joy             2
Joy/Sadness          2
Sadness/Surprise     2
Disgust/Surprise     2
Anger/Fear           2
Fear/Neutral         1
Surprise             1
Hope/Surprise        1
Disgust              1
Neutral              1
Disgust/Joy          1
Neutral/Surprise     1
Name: pred_all, dtype: int64

In [23]:
# binarize predictions
df_dev['pred_encoded'] = df_dev['pred_all'].apply( lambda x: get_target(x.split('/')) )

In [24]:
y_dev_encoded      = np.array( df_dev['target_encoded'].values.tolist() )
y_dev_pred_encoded = np.array( df_dev['pred_encoded'].values.tolist() )
labels = list(label2key.keys())
print( classification_report( y_dev_encoded, y_dev_pred_encoded, target_names=labels, digits=4 ) )

              precision    recall  f1-score   support

       Anger     0.5556    0.5263    0.5405        38
     Disgust     0.3443    0.8750    0.4941        24
        Fear     0.1667    1.0000    0.2857         8
        Hope     0.1923    0.9375    0.3191        16
         Joy     0.2000    0.5000    0.2857         2
     Neutral     0.6500    0.2407    0.3514        54
     Sadness     0.6463    0.9406    0.7661       101
    Surprise     0.1111    0.3333    0.1667         3

   micro avg     0.4307    0.7073    0.5354       246
   macro avg     0.3583    0.6692    0.4012       246
weighted avg     0.5483    0.7073    0.5578       246
 samples avg     0.4423    0.7019    0.5280       246



### If a followup question was used

In [36]:
# if a followup question was used - clean reponse
df_dev['pred'] = df_dev['pred_all'].apply( lambda x: x[3] )
print('Null values:\n', df_dev.isna().sum(), sep='')
df_dev['pred'].value_counts()

Null values:
article_id                    0
conversation_id               0
speaker_number                0
essay_id                      0
speaker_id                    0
essay                         0
essay_clean                   0
split                         0
gender                        0
education                     0
race                          0
age                           0
income                        0
emotion                       0
emotion_count                 0
char_length                   0
word_length                   0
target_encoded                0
article                       0
article_clean                 0
essay_clean_docs              0
essay_clean_spellchecked      0
article_clean_docs            0
article_clean_spellchecked    0
compare1                      0
compare2                      0
pred_all                      0
pred_encoded                  0
pred                          3
dtype: int64


Sadness             54
Fear/Sadness        31
Hope/Sadness        24
Anger/Sadness       23
Disgust/Sadness     19
Anger/Disgust        9
Hope                 8
Fear                 6
Disgust              5
Neutral/Sadness      4
Anger/Fear           4
Hope/Neutral         4
Disgust/Fear         3
Anger                2
Anger/Neutral        2
Neutral              2
Fear/Neutral         1
Disgust/Neutral      1
Sadness/Surprise     1
Joy                  1
Fear/Surprise        1
Name: pred, dtype: int64

In [42]:
# if a followup question was used - review why some predictions were NaNs
temp = df_dev[ df_dev['pred'].isna() ]
print(temp.index)
temp[['pred_all', 'pred']].values.tolist()

Int64Index([150, 167, 173], dtype='int64')


[[('Neutral',
   'Neutral',
   'I think the text is more related to Disappointment.',
   None),
  None],
 [('Sadness',
   'Sadness',
   "Yes, I am sure. The text expresses concern about children not being able to eat and the country's failure to provide basic necessities to its people, which is a sad situation.",
   None),
  None],
 [('Hope/Sadness',
   'Hope/Sadness',
   "Yes, I'm sure. The text expresses sympathy for the situation but also a sense of helplessness, and ends with a hopeful attitude towards the future.",
   None),
  None]]

In [43]:
# if a followup question was used - manually assign missing predictions
df_dev.at[150, 'pred'] = 'Neutral'
df_dev.at[167, 'pred'] = 'Sadness'
df_dev.at[173, 'pred'] = 'Hope/Sadness'

In [61]:
# binarize predictions
df_dev['pred_encoded'] = df_dev['pred_all'].apply( lambda x: get_target(x.split('/')) )

In [62]:
y_dev_encoded      = np.array( df_dev['target_encoded'].values.tolist() )
y_dev_pred_encoded = np.array( df_dev['pred_encoded'].values.tolist() )
labels = list(label2key.keys())
print( classification_report( y_dev_encoded, y_dev_pred_encoded, target_names=labels, digits=4 ) )

              precision    recall  f1-score   support

       Anger     0.6061    0.5263    0.5634        38
     Disgust     0.4054    0.6250    0.4918        24
        Fear     0.2083    0.6250    0.3125         8
        Hope     0.2500    0.5000    0.3333        16
         Joy     0.3333    0.5000    0.4000         2
     Neutral     0.6316    0.2222    0.3288        54
     Sadness     0.6905    0.8614    0.7665       101
    Surprise     0.0000    0.0000    0.0000         3

   micro avg     0.5362    0.6016    0.5670       246
   macro avg     0.3906    0.4825    0.3995       246
weighted avg     0.5810    0.6016    0.5570       246
 samples avg     0.5801    0.6202    0.5768       246



In [56]:
file = 'data/predictions_EMO.tsv'
with open(file, 'w', encoding='utf-8') as f:
    f.write('\n'.join(df_dev['pred_all'].tolist()))

# Results

Notes:
* Zero-shot with GPT-4 is less accurate than with ChatGPT
* Reason: GPT-4 is too eager to output the second emotion category, even when is it not required
* Several prompts tried to make sure GPT-4 "understands" not to include the second emotion if it is no relevant
* Few-shot learning may be a better way to go here

__Experiment 1__:  
General feeling - GPT-4 uses the second categotry too eagerly in the zero-shot mode. The results are worse than for ChatGPT. Temperature is 0.5 for this experiment, but GPT-4 is too eager to output the second emotion category even when temp=0 (Experiment 5)

_Prompt_:  
```
1 - As the first step, please process the essay delimited below with triple backticks and carefully analyze it for the presence of emotions.
2 - Then act as a helpful emotion classifier and classify the emotion in this essay as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy.
3 - You may add another emotion from the above list ONLY AND ONLY IF it is also relevant to the essay.
4 - Output only the relevant emotion and nothing else. If there are two relevant emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.

Essay: ```This is a text sample```


If the response doesn't contain any categories, the second prompt is used:  
'Are you sure about that? If yes, output the same emotion; if no, change the emotion'


              precision    recall  f1-score   support

       Anger     0.5227    0.6053    0.5610        38
     Disgust     0.4324    0.6667    0.5246        24
        Fear     0.1600    1.0000    0.2759         8
        Hope     0.2143    0.9375    0.3488        16
         Joy     0.2500    0.5000    0.3333         2
     Neutral     0.6522    0.2778    0.3896        54
     Sadness     0.6736    0.9604    0.7918       101
    Surprise     0.0476    0.3333    0.0833         3

   micro avg     0.4478    0.7154    0.5509       246
   macro avg     0.3691    0.6601    0.4135       246
weighted avg     0.5644    0.7154    0.5838       246
 samples avg     0.4720    0.7236    0.5521       246
```
_____

__Experiment 2__:  
Temperature = 0.5  
_Modified prompt by adding item 4_ (because of GPT-4's eagerness to add the second category):  
```
1 - As the first step, please process the essay delimited below with triple backticks and carefully analyze it for the presence of emotions.
2 - Then act as a helpful emotion classifier and classify the emotion in this essay as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy.
3 - You may add another emotion from the above list ONLY AND ONLY IF it is also relevant to the essay.
4 - Be mindful about the second emotion - add it only if it is really relevant to the essay.
5 - Output only the relevant emotion and nothing else. If there are two relevant emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.

Essay: ```This is a text sample```


If the response doesn't contain any categories, the second prompt is used:  
'Are you sure about that? If yes, output the same emotion; if no, change the emotion'

              precision    recall  f1-score   support

       Anger     0.5532    0.6842    0.6118        38
     Disgust     0.4146    0.7083    0.5231        24
        Fear     0.1346    0.8750    0.2333         8
        Hope     0.1899    0.9375    0.3158        16
         Joy     0.3333    0.5000    0.4000         2
     Neutral     0.6667    0.2593    0.3733        54
     Sadness     0.6759    0.9703    0.7967       101
    Surprise     0.1000    0.6667    0.1739         3

   micro avg     0.4412    0.7317    0.5505       246
   macro avg     0.3835    0.7002    0.4285       246
weighted avg     0.5704    0.7317    0.5881       246
 samples avg     0.4471    0.7332    0.5425       246
```
_____

__Experiment 3__  
Trying to understand why zero shot GPT-4 results are worse than ChatGPT, I removed the first instruction in the prompt to make it more terse and clear.  

Temperature = 0.5  

_Promtp_:  
```
1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy.
2 - You can add another emotion from the above list only and only if it is also relevant to the text below.
3 - Be mindful about the second emotion - add it only if it is really relevant to the text.
4 - Output only the relevant emotion and nothing else. If there are two relevant emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.

Text: ```This is a text sample```

If the response doesn't contain any categories, the second prompt is used:  
'Are you sure about that? If yes, output the same emotion; if no, change the emotion'

              precision    recall  f1-score   support

       Anger     0.5938    0.5000    0.5429        38
     Disgust     0.3529    0.7500    0.4800        24
        Fear     0.1067    1.0000    0.1928         8
        Hope     0.1831    0.8125    0.2989        16
         Joy     0.3333    0.5000    0.4000         2
     Neutral     0.6667    0.2963    0.4103        54
     Sadness     0.6857    0.9505    0.7967       101
    Surprise     0.1000    0.3333    0.1538         3

   micro avg     0.4236    0.6992    0.5276       246
   macro avg     0.3778    0.6428    0.4094       246
weighted avg     0.5733    0.6992    0.5787       246
 samples avg     0.4303    0.7067    0.5224       246
```

__Experiment 4__  
Trying to mitigate the eagerness to output the second emotion using a different prompt.  

Temperature = 0.5  

_Prompt_:  
```
1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but only if it is also relevant. If there are two relevant emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.
3 - Do not add the second emotion if it doesn't have a significant semantic weight in the text below.

Text: ```This is a text sample```

If the response doesn't contain any categories, the second prompt is used:  
'Are you sure about that? If yes, output the same emotion; if no, change the emotion'

              precision    recall  f1-score   support

       Anger     0.5526    0.5526    0.5526        38
     Disgust     0.3725    0.7917    0.5067        24
        Fear     0.1290    1.0000    0.2286         8
        Hope     0.1940    0.8125    0.3133        16
         Joy     0.2000    0.5000    0.2857         2
     Neutral     0.6667    0.2963    0.4103        54
     Sadness     0.6490    0.9703    0.7778       101
    Surprise     0.1667    0.3333    0.2222         3

   micro avg     0.4381    0.7195    0.5446       246
   macro avg     0.3663    0.6571    0.4121       246
weighted avg     0.5550    0.7195    0.5770       246
 samples avg     0.4495    0.7236    0.5401       246
```

__Experiment 5__  
Trying to mitigate the eagerness to output the second emotion using a different prompt.  

__Temperature = 0__  

_Prompt_:  
```
1 - Act as a helpful emotion classifier and classify the emotion in the text delimited below with tripple backticks as either Sadness, Neutral, Anger, Disgust, Fear, Hope, Surprise, or Joy. Output only this emotion and nothing elser
2 - You can add another emotion from the above list, but only if it is also relevant. If there are two relevant emotions: sort them alphabetically, concatenate them with a forward slash, and output only them and nothing else.
3 - Do not add the second emotion if it doesn't have a significant semantic weight in the text below.

Text: ```This is a text sample```

If the response doesn't contain any categories, the second prompt is used:  
'Are you sure about that? If yes, output the same emotion; if no, change the emotion'
              precision    recall  f1-score   support

       Anger     0.5556    0.5263    0.5405        38
     Disgust     0.3443    0.8750    0.4941        24
        Fear     0.1667    1.0000    0.2857         8
        Hope     0.1923    0.9375    0.3191        16
         Joy     0.2000    0.5000    0.2857         2
     Neutral     0.6500    0.2407    0.3514        54
     Sadness     0.6463    0.9406    0.7661       101
    Surprise     0.1111    0.3333    0.1667         3

   micro avg     0.4307    0.7073    0.5354       246
   macro avg     0.3583    0.6692    0.4012       246
weighted avg     0.5483    0.7073    0.5578       246
 samples avg     0.4423    0.7019    0.5280       246
```